## MoCo with clustering


### Prepare

Check GPU settings. A free GPU in Colab is <= Tesla P100. The log of the demo is based on a Tesla V100 from Google Cloud Platform.


In [1]:
from datetime import datetime
from functools import partial
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchvision.models import resnet
from tqdm import tqdm
import argparse
import json
import math
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
os.environ["CUDA_VISIBLE_DEVICES"]='0'

### Set arguments

In [2]:
parser = argparse.ArgumentParser(description='Train MoCo on Hypper Kvisia')

parser.add_argument('-a', '--arch', default='resnet18')

# lr: 0.06 for batch 512 (or 0.03 for batch 256)
parser.add_argument('--lr', '--learning-rate', default=0.03, type=float, metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--epochs', default=50, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--schedule', default=[120, 160], nargs='*', type=int, help='learning rate schedule (when to drop lr by 10x); does not take effect if --cos is on')
parser.add_argument('--cos', action='store_true', help='use cosine lr schedule')

parser.add_argument('--batch-size', default=128, type=int, metavar='N', help='mini-batch size')
parser.add_argument('--wd', default=5e-4, type=float, metavar='W', help='weight decay')

# moco specific configs:
parser.add_argument('--moco-dim', default=64, type=int, help='feature dimension')
parser.add_argument('--moco-k', default=2048, type=int, help='queue size; number of negative keys')
parser.add_argument('--moco-m', default=0.99, type=float, help='moco momentum of updating key encoder')
parser.add_argument('--moco-t', default=0.1, type=float, help='softmax temperature')

parser.add_argument('--bn-splits', default=8, type=int, help='simulate multi-gpu behavior of BatchNorm in one gpu; 1 is SyncBatchNorm in multi-gpu')

parser.add_argument('--symmetric', action='store_true', help='use a symmetric loss function that backprops to both crops')

# knn monitor
parser.add_argument('--knn-k', default=20, type=int, help='k in kNN monitor')
parser.add_argument('--knn-t', default=0.1, type=float, help='softmax temperature in kNN monitor; could be different with moco-t')

# utils
parser.add_argument('--resume', default='', type=str, metavar='PATH', help='path to latest checkpoint (default: none)')
parser.add_argument('--results-dir', default='', type=str, metavar='PATH', help='path to cache (default: none)')

'''
args = parser.parse_args()  # running in command line
'''
args = parser.parse_args('')  # running in ipynb

# set command line arguments here when running in ipynb
args.epochs = 100
args.cos = True
args.schedule = []  # cos in use
args.symmetric = False
if args.results_dir == '':
    args.results_dir = './static_weight-2048-logweight-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S-moco")

print(args)

Namespace(arch='resnet18', batch_size=128, bn_splits=8, cos=True, epochs=100, knn_k=20, knn_t=0.1, lr=0.03, moco_dim=64, moco_k=2048, moco_m=0.99, moco_t=0.1, results_dir='./static_weight-2048-logweight-2022-05-22-06-43-37-moco', resume='', schedule=[], symmetric=False, wd=0.0005)


### Define data loaders

In [3]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

# # data prepare
# train_data = CIFAR10Pair(root='data', train=True, transform=train_transform, download=True)
# train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=0, pin_memory=True, drop_last=True)

# memory_data = CIFAR10(root='data', train=True, transform=test_transform, download=True)
# memory_loader = DataLoader(memory_data, batch_size=args.batch_size, shuffle=False, num_workers=0, pin_memory=True)

# test_data = CIFAR10(root='data', train=False, transform=test_transform, download=True)
# test_loader = DataLoader(test_data, batch_size=args.batch_size, shuffle=False, num_workers=0, pin_memory=True)

import csv
from torch.utils.data import Dataset

def split_name(address):
    n = address.rfind("\\")
    dot = address[:n].rfind("\\")
    return address[dot+1:n]

class MyDataset(Dataset):
    def __init__(self, data_dir, train=True, transform=None, isLabel=True):
        """
        My Dataset for CAD with BUSI dataset
            param data_dir: str, path for the dataset
            param train: whether this is defined for training or testing
            param transform: torch.transform, data pre-processing pipeline
        """
        ### Begin your code ###
        super(MyDataset,self).__init__()
        self.train = train
        self.transform = transform
        self.isLabel = isLabel
        
        if isLabel==False:
            self.root = data_dir
        else:
            self.root = os.path.join(data_dir,"unsupervised")
            
            # filepath_csv = os.path.join(data_dir,"image-labels.csv")
            self.class_map = {"polyps":0, "ulcerative-colitis-grade-1":1, "ulcerative-colitis-grade-2":2, "ulcerative-colitis-grade-3":3}
            self.classes = list(self.class_map.keys())
            # with open(filepath_csv) as f:
            #     reader = csv.reader(f)
            #     header_row = next(reader)
                
            #     for row in reader:
            #         name = row[0]
            #         class_ = row[2]
            #         if type(class_map.get(class_))==int:
            #             self.label_map[name] = class_map[class_]
        
        self.filepaths = []
        self.data = []
        self.targets = []
        
        self.get_filename(self.root)
        self.length = len(self.filepaths)
        self.get_data()
        self.filepaths = []
        
        ### End your code ###
        
    
    def __getitem__(self, index):
        if self.isLabel == False:
            img = self.data[index]

            if self.transform is not None:
                im_1 = self.transform(img)
                im_2 = self.transform(img)
            return im_1, im_2
        else: 
            img = self.data[index]
            img = self.transform(img)
            return img, self.targets[index]


    def __len__(self): 
        '''return the size of the dataset'''
        ### Begin your code ###
        return len(self.data)
        ### End your code ###
    
    def get_filename(self, rootDir):
        for root,dirs,files in os.walk(rootDir):
            for file in files:
                file_path = os.path.join(root, file)
                self.filepaths.append(file_path)
        
    def get_data(self):
        '''
        Load the dataset and store it in your own data structure(s)
        '''
        ### Begin your code ###
        length = len(self.filepaths)
        
        index_list = torch.arange(0,length)
        index_list = torch.randperm(index_list.size(0))
        
        for i in tqdm(range(length)):
            index = index_list[i].item()
            filepath = self.filepaths[index]

            if self.isLabel == False:
                image = Image.open(filepath).resize((128,128))
                self.data.append(image.copy())
                image.close()

            else:
                label_name = split_name(filepath)
                label = self.class_map[label_name]
                image = Image.open(filepath).resize((128,128))
                self.data.append(image.copy())
                image.close()
                self.targets.append(label)
        
        total_length = len(self.data)
        if self.isLabel == False:
            begin = 0
            end = total_length
        else: 
            threshold = int(total_length * 0.8)
            if self.train == True:
                begin = 0
                end = threshold
            else:
                begin = threshold
                end = total_length

        self.length = end - begin        
        self.data = self.data[begin:end]
        self.targets = self.targets[begin:end]
            
# data prepare
# train_dir = r"H:\csc4080data\unlabeled"
# test_dir = r"H:\csc4080data\labeled-images"
train_dir = r"G:\Test\data\unlabeled-images"
test_dir = r"G:\Test\data\labeled-images"

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])


In [4]:
train_data = MyDataset(train_dir, train=True, transform=train_transform, isLabel=False)
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=0, pin_memory=True, drop_last=True)

100%|██████████| 12471/12471 [01:35<00:00, 129.98it/s]


In [5]:
memory_data = MyDataset(test_dir, train=True, transform=test_transform,isLabel=True)
memory_loader = DataLoader(memory_data, batch_size=args.batch_size, shuffle=False, num_workers=0, pin_memory=True)

100%|██████████| 1878/1878 [00:07<00:00, 268.08it/s]


In [6]:
test_data = MyDataset(test_dir, train=False, transform=test_transform,isLabel=True)
test_loader = DataLoader(test_data, batch_size=args.batch_size, shuffle=False, num_workers=0, pin_memory=True)

100%|██████████| 1878/1878 [00:06<00:00, 269.77it/s]


### Define base encoder

In [7]:
# SplitBatchNorm: simulate multi-gpu behavior of BatchNorm in one gpu by splitting alone the batch dimension
# implementation adapted from https://github.com/davidcpage/cifar10-fast/blob/master/torch_backend.py
class SplitBatchNorm(nn.BatchNorm2d):
    def __init__(self, num_features, num_splits, **kw):
        super().__init__(num_features, **kw)
        self.num_splits = num_splits
        
    def forward(self, input):
        N, C, H, W = input.shape
        if self.training or not self.track_running_stats:
            running_mean_split = self.running_mean.repeat(self.num_splits)
            running_var_split = self.running_var.repeat(self.num_splits)
            outcome = nn.functional.batch_norm(
                input.view(-1, C * self.num_splits, H, W), running_mean_split, running_var_split, 
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H, W)
            self.running_mean.data.copy_(running_mean_split.view(self.num_splits, C).mean(dim=0))
            self.running_var.data.copy_(running_var_split.view(self.num_splits, C).mean(dim=0))
            return outcome
        else:
            return nn.functional.batch_norm(
                input, self.running_mean, self.running_var, 
                self.weight, self.bias, False, self.momentum, self.eps)

class ModelBase(nn.Module):

    def __init__(self, feature_dim=128, arch=None, bn_splits=16):
        super(ModelBase, self).__init__()

        # use split batchnorm
        norm_layer = partial(SplitBatchNorm, num_splits=bn_splits) if bn_splits > 1 else nn.BatchNorm2d
        resnet_arch = getattr(resnet, arch)
        net = resnet_arch(num_classes=feature_dim, norm_layer=norm_layer)

        self.net = []
        for name, module in net.named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if isinstance(module, nn.MaxPool2d):
                continue
            if isinstance(module, nn.Linear):
                self.net.append(nn.Flatten(1))
            self.net.append(module)

        self.net = nn.Sequential(*self.net)

    def forward(self, x):
        x = self.net(x)
        # note: not normalized here
        return x

### Define MoCo wrapper

In [8]:
def my_cross_entropy(input, target, reduction="mean"):
    exp = torch.exp(input)
    tmp1 = exp.gather(1, target.unsqueeze(-1)).squeeze()
    tmp2 = exp.sum(1)
    #softmax = tmp1 / tmp2
    softmax = 1. / tmp2
    log = -torch.log(softmax)
    if reduction == "mean": return log.mean()
    else: return log.sum()
    
def my_loss_1(l_pos, l_neg, logits, T, reduction="mean"):
    b = torch.exp(l_neg/T)
    log = -torch.log(3./(3. + b.sum(1)))
    if reduction == "mean": return log.mean()
    else: return log.sum()
    
def my_loss_3(l_pos, l_neg, logits, T, weight, reduction="mean"):
    tmp1 = torch.exp(l_pos/T)
    
    exp_neg = torch.exp(l_neg/T)
    #print(exp_neg.shape)
    #tmp2 = exp_neg.sum(1)
    #print(min(weight))
    tmp2 = torch.mm(exp_neg, weight)
    
    softmax = tmp1 / (tmp1 + tmp2)
    log = -torch.log(softmax)
    
#    b = torch.exp(l_neg/T)
#    log = -torch.log(3./(3. + b.sum(1)))
    if reduction == "mean": return log.mean(), tmp1.mean(), tmp2.mean()
    else: return log.sum()

In [9]:
class KMEANS:
    def __init__(self, n_clusters, centers, num_batch, max_iter=3, verbose=True,device = torch.device("cuda:0")):
        self.n_clusters = n_clusters
        self.centers = centers
        self.max_iter = max_iter
        self.statistics = torch.ones(centers.shape[1],1)
        self.old_stats = torch.ones(centers.shape[1],1)
        self.num_batch = num_batch # number of batches within one epoch
        self.statistics_mix = self.statistics
        self.batch_counter = 0
        self.variation = torch.Tensor([float("Inf")]).to(device)
        self.device = device

    def fit(self, x):
        count = 0
        while True:
            # mark clusters 
            self.nearest_center(x)
            # update centroids
            self.update_center(x)
            if torch.abs(self.variation) < 1e-3 and self.max_iter is None:
                break
            elif self.max_iter is not None and count == self.max_iter:
                break
            count += 1
        if self.batch_counter == self.num_batch-1:
            self.old_stats = self.statistics
            self.batch_counter = 0
            self.statistics = torch.ones(self.statistics.shape[0],1)
            self.centers = nn.functional.normalize(self.centers)
            print(torch.sum(self.statistics_mix != 1))
        else:
            self.batch_counter += 1
        # print("BC:", self.batch_counter)
        return self.centers

    def nearest_center(self, x):
        labels = torch.empty((x.shape[0],))
        dists = torch.empty((0, self.n_clusters))
        #for i, sample in enumerate(x):
            #dist = torch.sum(torch.mul(sample - self.centers, sample - self.centers), (1))
        dist = torch.einsum('bd,dc->bc', [x, self.centers])
        #print(dist.shape)
        labels = torch.argmax(dist,1)
        #dists = torch.cat([dists, dist.unsqueeze(0)], (0))
        self.labels = labels
        #print(labels.shape)

    def update_center(self, x):
        frac = 1.0 * self.batch_counter/(self.num_batch-1)
        self.statistics_mix = self.statistics * frac + self.old_stats * (1-frac)
        for i in range(self.n_clusters):
            mask = self.labels == i
            cluster_samples = x[mask]
            #centers = torch.cat([centers, torch.mean(cluster_samples, (0)).unsqueeze(0)], (0))
            if cluster_samples.shape[0] > 0:
                #self.centers[:,i] = (torch.sum(cluster_samples, (0))+ self.centers[:,i]*self.statistics_mix[i,0]) /(cluster_samples.shape[0]+self.statistics_mix[i,0])
                #self.centers[:,i] = (torch.sum(cluster_samples, (0))+ self.centers[:,i]*self.statistics[i,0]) /(1.0 * (cluster_samples.shape[0]+self.statistics[i,0]))
                self.centers[:,i] = torch.mean(cluster_samples, (0))
                self.statistics[i,0] += cluster_samples.shape[0]
        #self.centers = centers
    def get_weight(self):
        return self.statistics_mix.to(self.device)


In [10]:
class ModelMoCo(nn.Module):
    def __init__(self, num_batch, dim=128, K=4096, m=0.99, T=0.1, arch='resnet18', bn_splits=8, symmetric=True):
        super(ModelMoCo, self).__init__()

        self.K = K
        self.m = m
        self.T = T
        self.symmetric = symmetric

        # create the encoders
        self.encoder_q = ModelBase(feature_dim=dim, arch=arch, bn_splits=bn_splits)
        self.encoder_k = ModelBase(feature_dim=dim, arch=arch, bn_splits=bn_splits)

        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)  # initialize
            param_k.requires_grad = False  # not update by gradient

        # create the queue
        self.queue = torch.rand(dim, K)
        self.queue = nn.functional.normalize(self.queue, dim=0).cuda()
        self.kmeans = KMEANS(K, self.queue, num_batch, max_iter=3)
        #self.register_buffer("queue", torch.randn(dim, K))
        #self.queue = nn.functional.normalize(self.queue, dim=0)
        #self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))
        self.statistic = torch.ones(K,1)

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Momentum update of the key encoder
        """
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1. - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]
        print("keys.shape:", keys.shape)
        self.queue = self.kmeans.fit(keys.clone().detach())
        #print(keys.shape)
        self.queue = nn.functional.normalize(self.queue, dim=0)
        #ptr = int(self.queue_ptr)
        #assert self.K % batch_size == 0  # for simplicity

        # replace the keys at ptr (dequeue and enqueue)
        #self.queue[:, ptr:ptr + batch_size] = keys.t()  # transpose
        #ptr = (ptr + batch_size) % self.K  # move pointer

        #self.queue_ptr[0] = ptr

    @torch.no_grad()
    def _batch_shuffle_single_gpu(self, x):
        """
        Batch shuffle, for making use of BatchNorm.
        """
        # random shuffle index
        idx_shuffle = torch.randperm(x.shape[0]).cuda()

        # index for restoring
        idx_unshuffle = torch.argsort(idx_shuffle)

        return x[idx_shuffle], idx_unshuffle

    @torch.no_grad()
    def _batch_unshuffle_single_gpu(self, x, idx_unshuffle):
        """
        Undo batch shuffle.
        """
        return x[idx_unshuffle]

    def contrastive_loss(self, im_q, im_k):
        # compute query features
        q = self.encoder_q(im_q)  # queries: NxC
        q = nn.functional.normalize(q, dim=1)  # already normalized

        # compute key features
        with torch.no_grad():  # no gradient to keys
            # shuffle for making use of BN
            im_k_, idx_unshuffle = self._batch_shuffle_single_gpu(im_k)

            k = self.encoder_k(im_k_)  # keys: NxC
            k = nn.functional.normalize(k, dim=1)  # already normalized

            # undo shuffle
            k = self._batch_unshuffle_single_gpu(k, idx_unshuffle)

        # compute logits
        # Einstein sum is more intuitive
        # positive logits: Nx1
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        # negative logits: NxK
        l_neg = torch.einsum('nc,ck->nk', [q, self.queue.clone().detach()])

        # logits: Nx(1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators
        labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()
        #print(logits.shape, labels.shape)
        #loss = my_cross_entropy(logits, labels)
        weight = self.kmeans.get_weight()
        loss, tmp1, tmp2 = my_loss_3(l_pos, l_neg, logits, self.T, weight)
        return loss, q, k, tmp1, tmp2

    def forward(self, im1, im2):
        """
        Input:
            im_q: a batch of query images
            im_k: a batch of key images
        Output:
            loss
        """

        # update the key encoder
        with torch.no_grad():  # no gradient to keys
            self._momentum_update_key_encoder()

        # compute loss
        if self.symmetric:  # asymmetric loss
            loss_12, q1, k2 = self.contrastive_loss(im1, im2)
            loss_21, q2, k1 = self.contrastive_loss(im2, im1)
            loss = loss_12 + loss_21
            k = torch.cat([k1, k2], dim=0)
        else:  # asymmetric loss
            loss, q, k, tmp1, tmp2 = self.contrastive_loss(im1, im2)

        self._dequeue_and_enqueue(k)

        return loss, tmp1, tmp2

# create model
model = ModelMoCo(
        len(train_loader),
        dim=args.moco_dim,
        K=args.moco_k,
        m=args.moco_m,
        T=args.moco_t,
        arch=args.arch,
        bn_splits=args.bn_splits,
        symmetric=args.symmetric,
    ).cuda()

### Define train/test



In [11]:
# train for one epoch
def train(net, data_loader, train_optimizer, epoch, args):
    net.train()
    adjust_learning_rate(optimizer, epoch, args)
    
    total_loss, total_num, total_tmp1, total_tmp2, train_bar = 0.0, 0, 0.0, 0.0, tqdm(data_loader)
    for im_1, im_2 in train_bar:
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

        loss,tmp1,tmp2 = net(im_1, im_2)
        
        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        total_tmp1 += tmp1.item() * data_loader.batch_size
        total_tmp2 += tmp2.item() * data_loader.batch_size
        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, args.epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num, total_tmp1 / total_num, total_tmp2 / total_num

# lr scheduler for training
def adjust_learning_rate(optimizer, epoch, args):
    """Decay the learning rate based on schedule"""
    lr = args.lr
    if args.cos:  # cosine lr schedule
        lr *= 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    else:  # stepwise lr schedule
        for milestone in args.schedule:
            lr *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [12]:
# test using a knn monitor
def test(net, memory_data_loader, test_data_loader, epoch, args):
    net.eval()
    classes = len(memory_data_loader.dataset.classes)
    total_top1, total_top5, total_num, feature_bank = 0.0, 0.0, 0, []
    with torch.no_grad():
        # generate feature bank
        for data, target in tqdm(memory_data_loader, desc='Feature extracting'):
            feature = net(data.cuda(non_blocking=True))
            feature = F.normalize(feature, dim=1)
            feature_bank.append(feature)
        # [D, N]
        feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
        # [N]
        feature_labels = torch.tensor(memory_data_loader.dataset.targets, device=feature_bank.device)
        # loop test data to predict the label by weighted knn search
        test_bar = tqdm(test_data_loader)
        for data, target in test_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            feature = net(data)
            feature = F.normalize(feature, dim=1)
            
            pred_labels = knn_predict(feature, feature_bank, feature_labels, classes, args.knn_k, args.knn_t)

            total_num += data.size(0)
            total_top1 += (pred_labels[:, 0] == target).float().sum().item()
            test_bar.set_description('Test Epoch: [{}/{}] Acc@1:{:.2f}%'.format(epoch, args.epochs, total_top1 / total_num * 100))

    return total_top1 / total_num * 100

# knn monitor as in InstDisc https://arxiv.org/abs/1805.01978
# implementation follows http://github.com/zhirongw/lemniscate.pytorch and https://github.com/leftthomas/SimCLR
def knn_predict(feature, feature_bank, feature_labels, classes, knn_k, knn_t):
    # compute cos similarity between each feature vector and feature bank ---> [B, N]
    sim_matrix = torch.mm(feature, feature_bank)
    # [B, K]
    sim_weight, sim_indices = sim_matrix.topk(k=knn_k, dim=-1)
    # [B, K]
    sim_labels = torch.gather(feature_labels.expand(feature.size(0), -1), dim=-1, index=sim_indices)
    sim_weight = (sim_weight / knn_t).exp()

    # counts for each class
    one_hot_label = torch.zeros(feature.size(0) * knn_k, classes, device=sim_labels.device)
    # [B*K, C]
    one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
    # weighted score ---> [B, C]
    pred_scores = torch.sum(one_hot_label.view(feature.size(0), -1, classes) * sim_weight.unsqueeze(dim=-1), dim=1)

    pred_labels = pred_scores.argsort(dim=-1, descending=True)
    return pred_labels

In [13]:
# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.wd, momentum=0.9)

# load model if resume
epoch_start = 1
if args.resume is not '':
    checkpoint = torch.load(args.resume)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = checkpoint['epoch'] + 1
    print('Loaded from: {}'.format(args.resume))

# logging
results = {'train_loss': [], 'test_acc@1': [], 'a':[], 'b':[]}
if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)
# dump args
with open(args.results_dir + '/args.json', 'w') as fid:
    json.dump(args.__dict__, fid, indent=2)

# training loop
for epoch in range(epoch_start, args.epochs + 1):
    train_loss, tmp1, tmp2 = train(model, train_loader, optimizer, epoch, args)
    results['train_loss'].append(train_loss)
    results['a'].append(tmp1)
    results['b'].append(tmp2)
    test_acc_1 = test(model.encoder_q, memory_loader, test_loader, epoch, args)
    results['test_acc@1'].append(test_acc_1)
    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(epoch_start, epoch + 1))
    data_frame.to_csv(args.results_dir + '/log-2048.csv', index_label='epoch')
    # save model
    torch.save({'epoch': epoch, 'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_last.pth')

<>:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12252\304079192.py:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if args.resume is not '':
  0%|          | 0/97 [00:00<?, ?it/s]

keys.shape: torch.Size([128, 64])


Train Epoch: [1/100], lr: 0.029993, Loss: 0.1079:   1%|          | 1/97 [00:03<05:05,  3.19s/it]

keys.shape: torch.Size([128, 64])


Train Epoch: [1/100], lr: 0.029993, Loss: 1.7636:   2%|▏         | 2/97 [00:04<02:59,  1.89s/it]

...

keys.shape: torch.Size([128, 64])


Train Epoch: [11/100], lr: 0.029113, Loss: 9.2582:  51%|█████     | 49/97 [00:47<00:46,  1.03it/s]

keys.shape: torch.Size([128, 64])


Train Epoch: [11/100], lr: 0.029113, Loss: 9.2582:  51%|█████     | 49/97 [00:48<00:47,  1.02it/s]


KeyboardInterrupt: 

### Start training